In [1]:
import re
import codecs
import random
from pythainlp.corpus.common import thai_stopwords
from deeple_preprocessor.tokenize import newmm_tokenize
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm
from pythainlp.tag import pos_tag

In [2]:
stopwords = list(thai_stopwords())
#จัดการประโยคซ้ำ
data_not=[]

def remove_emojis(text):
    pattern = re.compile(r"[\u0E00-\u0E7Fa-zA-Z0-9' \n\-!$%^&*()_+|~=`{}\[\]:'<>?,.#\/]")
    return ''.join(pattern.findall(text))
def normalize_space(text):
    text = text.replace('\t', '')
    text = text.replace('\u200b', '')
    text = text.replace('   ', ' ')
    text = text.replace('  ', ' ')
    return text.strip()

def Unique(p):
    text=re.sub("<[^>]*>","",p)
    text=re.sub("\[(.*?)\]","",text)
    text=re.sub("\[\/(.*?)\]","",text)
    if text not in data_not:
        data_not.append(text)
        return True
    else:
        return False
# เตรียมตัวตัด tag ด้วย re
pattern = r'\[(.*?)\](.*?)\[\/(.*?)\]'
tokenizer = RegexpTokenizer(pattern) # ใช้ nltk.tokenize.RegexpTokenizer เพื่อตัด [TIME]8.00[/TIME] ให้เป็น ('TIME','ไง','TIME')
# จัดการกับ tag ที่ไม่ได้ tag
def toolner_to_tag(text):
    text=text.strip().replace("FACILITY","LOCATION").replace("[AGO]","").replace("[/AGO]","").replace("[T]","").replace("[/T]","")
    text=re.sub("<[^>]*>","",text)
    text=re.sub("(\[\/(.*?)\])","\\1***",text)#.replace('(\[(.*?)\])','***\\1')# text.replace('>','>***') # ตัดการกับพวกไม่มี tag word
    text=re.sub("(\[\w+\])","***\\1",text)
    text2=[]
    for i in text.split('***'):
        if "[" in i:
            text2.append(i)
        else:
            text2.append("[word]"+i+"[/word]")
    text="".join(text2)#re.sub("[word][/word]","","".join(text2))
    return text.replace("[word][/word]","")
# แปลง text ให้เป็น conll2002
def text2conll2002(text,pos=True):
    """
    ใช้แปลงข้อความให้กลายเป็น conll2002
    """
    text=toolner_to_tag(text)
    text=text.replace("''",'"')
    text=text.replace("’",'"').replace("‘",'"')#.replace('"',"")
    tag=tokenizer.tokenize(text)
    j=0
    conll2002=""
    for tagopen,text,tagclose in tag:
        word_cut=newmm_tokenize(text) # ใช้ตัวตัดคำ newmm
        i=0
        txt5=""
        while i<len(word_cut):
            if word_cut[i]=="''" or word_cut[i]=='"':pass
            elif i==0 and tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'B-'+tagopen
            elif tagopen!='word':
                txt5+=word_cut[i]
                txt5+='\t'+'I-'+tagopen
            else:
                txt5+=word_cut[i]
                txt5+='\t'+'O'
            txt5+='\n'
            #j+=1
            i+=1
        conll2002+=txt5
    if pos==False:
        return conll2002
    return postag(conll2002)
# ใช้สำหรับกำกับ pos tag เพื่อใช้กับ NER
# print(text2conll2002(t,pos=False))
def postag(text):
    listtxt=[i for i in text.split('\n') if i!='']
    list_word=[]
    for data in listtxt:
        list_word.append(data.split('\t')[0])
    #print(text)
    list_word=pos_tag(list_word,engine="perceptron", corpus="orchid_ud")
    text=""
    i=0
    for data in listtxt:
        text+=data.split('\t')[0]+'\t'+list_word[i][1]+'\t'+data.split('\t')[1]+'\n'
        i+=1
    return text
# เขียนไฟล์ข้อมูล conll2002
def write_conll2002(file_name,data):
    """
    ใช้สำหรับเขียนไฟล์
    """
    with codecs.open(file_name, "w", "utf-8-sig") as temp:
        temp.write(data)
    return True
# อ่านข้อมูลจากไฟล์
def get_data(fileopen):
    """
    สำหรับใช้อ่านทั้งหมดทั้งในไฟล์ทีละรรทัดออกมาเป็น list
    """
    with codecs.open(fileopen, 'r',encoding='utf-8-sig') as f:
        lines = f.read().splitlines()
    return [a for a in tqdm(lines) if Unique(a)] # เอาไม่ซ้ำกัน

def alldata(lists):
    text=""
    for data in tqdm(lists):
        text+=text2conll2002(data)
        text+='\n'
    return text

def alldata_list(lists):
    data_all=[]
    for data in lists:
        data_num=[]
        try:
            txt=text2conll2002(data,pos=True).split('\n')
            for d in txt:
                tt=d.split('\t')
                if d!="":
                    if len(tt)==3:
                        data_num.append((tt[0],tt[1],tt[2]))
                    else:
                        data_num.append((tt[0],tt[1]))
            #print(data_num)
            data_all.append(data_num)
        except:
            print(data)
    #print(data_all)
    return data_all

def alldata_list_str(lists):
    string=""
    for data in lists:
        string1=""
        for j in data:
            string1+=j[0]+"	"+j[1]+"	"+j[2]+"\n"
        string1+="\n"
        string+=string1
    return string

def get_data_tag(listd):
    list_all=[]
    c=[]
    for i in listd:
        if i !='':
            c.append((i.split("\t")[0],i.split("\t")[1],i.split("\t")[2]))
        else:
            list_all.append(c)
            c=[]
    return list_all
def getall(lista):
    ll=[]
    for i in tqdm(lista):
        o=True
        for j in ll:
            if re.sub("\[(.*?)\]","",i)==re.sub("\[(.*?)\]","",j):
                o=False
                break
        if o==True:
            ll.append(normalize_space(remove_emojis(i)))
    return ll

In [3]:
data1=getall(get_data("dataset/generated_ner.txt"))
data_not = []
datatofile=alldata_list(data1)

100%|██████████| 8451/8451 [03:34<00:00, 39.38it/s] 


In [4]:
import dill
with open("dataset/ner.data", "wb") as dill_file:
    dill.dump(datatofile, dill_file)

# Validate corrupted data

In [5]:
for i in data1:
    pos = {j.start():['s', i[j.start():j.end()].replace('[', '').replace(']', '').replace('/', '')] 
           for j in re.finditer('\[.*?\]', i) if '/' not in i[j.start():j.end()]}
    pos.update({j.start():['e', i[j.start():j.end()].replace('[', '').replace(']', '').replace('/', '')]
                for j in re.finditer('\[.*?\]', i) if '/' in i[j.start():j.end()]})
    pos = sorted(pos.items())
    
    count = 0
    current = ''
    for k in pos:
        if(k[1][0] == 's'):
            count+=1
            if(current == ''):
                current = k[1][1]
            else:
                current = 'b'
        else:
            count-=1
            if(current == k[1][1]):
                current = ''
            else:
                current = 'b'
        if(current == 'b' or (count > 1 or count < 0)):
            print(i[max(k[0]-10, 0):min(k[0]+10, len(i))])
            print('---------------------')
            break

E]16.00 น.[/DATE] โด
---------------------
E]16.00 น.[/DATE] โด
---------------------
ม่ยืนยัน] [ORGANIZAT
---------------------
